In [2]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models.ddpg import DDPG
from models.model import OUNoise
from env.TradeEnv import TradeEnv

In [3]:
file_path = '../data/processed/'
df_macro = pd.read_csv(file_path + 'macro_data.csv')
df_stock = pd.read_csv(file_path + 'stock_data.csv')
df_factor = pd.read_csv(file_path + 'factor_data.csv')

env = TradeEnv(df_macro=df_macro, df_factor=df_factor, df_stock=df_stock)

Initialize trade environment...
Aligning df_macro, df_stock and df_factor...
df_macro shape:  (2674, 54)
df_stock shape:  (2674, 470)
df_factor shape:  (2674, 470)


In [6]:
agent = DDPG(env=env)

batch_size = 128
rewards = []
avg_rewards = []

for episode in range(50):
    state = env.reset()
    # noise.reset()
    episode_reward = 0
    
    for i in range(5000):
        action = agent.get_action(state)
        # action = noise.get_action(action, step)
        next_state, reward, terminated = env.step(action) 
        if not terminated:
            agent.memory.push(state, action, reward, next_state, terminated)
        
            state = next_state
            episode_reward += reward

        if len(agent.memory) > 1e4:
            agent.update(batch_size) 
            if (i+1) % 100 == 0:
                print('{}, {:>4d}: {:.7f}'.format(episode, i, episode_reward / (i+1)))

        if terminated:
            # sys.stdout.write("episode: {}, reward: {}, average _reward: {} \n".format(episode, np.round(episode_reward, decimals=2), np.mean(rewards[-10:])))
            break

    rewards.append(episode_reward)
    avg_rewards.append(np.mean(rewards[-10:]))

4, 1400: -0.0003080
4, 1500: -0.0003111
4, 1600: -0.0003146
4, 1700: -0.0003080
4, 1800: -0.0003083
4, 1900: -0.0003193
4, 2000: -0.0003233
4, 2100: -0.0003324
4, 2200: -0.0003393
4, 2300: -0.0003437
4, 2400: -0.0003564
4, 2500: -0.0003611
4, 2600: -0.0003653
5,    0: -0.0010000
5,  100: -0.0005678
5,  200: -0.0006067
5,  300: -0.0005832
5,  400: -0.0005889
5,  500: -0.0005935
5,  600: -0.0005534
5,  700: -0.0005477
5,  800: -0.0005510
5,  900: -0.0005393
5, 1000: -0.0005406
5, 1100: -0.0005452
5, 1200: -0.0005225
5, 1300: -0.0005007
5, 1400: -0.0004965
5, 1500: -0.0004943
5, 1600: -0.0004966
5, 1700: -0.0004930
5, 1800: -0.0004919
5, 1900: -0.0004866
5, 2000: -0.0004764
5, 2100: -0.0004687
5, 2200: -0.0004666
5, 2300: -0.0004565
5, 2400: -0.0004530
5, 2500: -0.0004453
5, 2600: -0.0004355
6,    0: -0.0010000
6,  100: -0.0002880
6,  200: -0.0003144
6,  300: -0.0002725
6,  400: -0.0002455
6,  500: -0.0002495
6,  600: -0.0002417
6,  700: -0.0002414
6,  800: -0.0002418
6,  900: -0.0002357


In [ ]:
plt.plot(rewards)
plt.plot(avg_rewards)
plt.plot()
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.show()